In [49]:
from openpyxl import load_workbook
from datetime import datetime
import pandas as pd
import re

# 1) Reading excel file using openpyxl

In [50]:
# Load the workbook and select the worksheet
file_path = 'data.xlsx'
wb = load_workbook(file_path, data_only=True)
ws = wb.active

# Create an empty dictionary to hold the data
data = {}

# Fetch the headers
for cell in ws[1]:
    data[cell.value] = []

# Initialize a dictionary to store date formats for columns
date_formats = {}



# Iterate over columns to find the first non-null value in each
for col in ws.iter_cols(min_row=2, values_only=False):
   
    header = ws.cell(row=1, column=col[0].column).value  # Get the column header
    for cell in col:
        if cell.value is not None:  # Check for the first non-null cell
            if cell.is_date:
                date_formats[header] = cell.number_format  # Store date format
            else:
                date_formats[header] = None  # No special format for non-date columns
            break  # Stop after finding the first non-null value

# Fetch the data for all rows without applying formats to the DataFrame
for row in ws.iter_rows(min_row=2, values_only=False):
    for cell in row:
        header = ws.cell(row=1, column=cell.column).value
        data[header].append(cell.value)

# Convert the dictionary to a pandas DataFrame (without storing formats)
df = pd.DataFrame(data)

# Print the date formats stored in the dictionary
print("Date columns and their formats:", date_formats)


Date columns and their formats: {'Ord ID': None, 'customer name': None, 'Purchase Date': None, 'Amount': None, 'Return Date': None, 'DoJ': None, 'doB': None, 'ord_date': None, 'Date': None, 'i am daTE shreedhar': None, 'dates': None, "date's": None, 'date1': None, 'date_with_date_type': 'yyyy\\-mm\\-dd\\ hh:mm:ss', 'date_with_date_type_2': None, 'temp_dates': 'mm-dd-yy', 'temp_dates_2': None, 'temp_dates_3': '[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy', 'temp_dates_4': None}


In [51]:
date_formats

{'Ord ID': None,
 'customer name': None,
 'Purchase Date': None,
 'Amount': None,
 'Return Date': None,
 'DoJ': None,
 'doB': None,
 'ord_date': None,
 'Date': None,
 'i am daTE shreedhar': None,
 'dates': None,
 "date's": None,
 'date1': None,
 'date_with_date_type': 'yyyy\\-mm\\-dd\\ hh:mm:ss',
 'date_with_date_type_2': None,
 'temp_dates': 'mm-dd-yy',
 'temp_dates_2': None,
 'temp_dates_3': '[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy',
 'temp_dates_4': None}

In [52]:
date_type_columns=dict()

In [53]:
for col, formats in date_formats.items():
    if formats != None:
        date_type_columns[col] = formats
date_type_columns

{'date_with_date_type': 'yyyy\\-mm\\-dd\\ hh:mm:ss',
 'temp_dates': 'mm-dd-yy',
 'temp_dates_3': '[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'}

In [54]:
date_data_type_columns = list(date_type_columns.keys())
date_data_type_columns

['date_with_date_type', 'temp_dates', 'temp_dates_3']

In [55]:
df

,Ord ID,customer name,Purchase Date,Amount,Return Date,DoJ,doB,ord_date,Date,i am daTE shreedhar,...,date1,date_1,date_with_date_type,date_with_date_type_2,x-y_z-date,my_x-y-date-is,temp_dates,temp_dates_2,temp_dates_3,temp_dates_4
0,1,jay,14-10-2021,100,2022-13-12,2015-10-13,13-10-2002,1-13-2020,1-1-2020,1-1-2020,...,None,None,2024-09-13 12:28:36.614,2024-09-13 12:28:36.614943,None,None,2024-12-13,12-13-2024,2024-12-13,12-13-2024 12:28:37
1,2,shree,13-2-2020,200,2021-10-11,2016-10-14,14-10-2002,12-10-2020,2-1-2020,None,...,None,None,2024-09-14 12:28:36.614,2024-09-14 12:28:36.614943,None,None,2024-12-14,12-13-2024,2024-12-14,12-13-2024 12:28:38
2,3,ram,1-10-2019,300,2020-1-1,2017-10-16,17-10-2002,5-10-2021,3-10-2020,2-1-2020,...,10-12-2002,None,2024-09-15 12:28:36.614,2024-09-15 12:28:36.614943,None,None,2024-12-15,12-13-2024,2024-12-15,12-13-2024 12:28:39


In [56]:
df.dtypes
# ⭐
#Reading all column w.r.t there data type,,,,not reading them as object or string and other columns are read as object or string

Ord ID                            int64
customer name                    object
Purchase Date                    object
Amount                            int64
Return Date                      object
DoJ                              object
doB                              object
ord_date                         object
Date                             object
i am daTE shreedhar              object
dates                            object
date's                           object
date1                            object
date_1                           object
date_with_date_type      datetime64[ns]
date_with_date_type_2            object
x-y_z-date                       object
my_x-y-date-is                   object
temp_dates               datetime64[ns]
temp_dates_2                     object
temp_dates_3             datetime64[ns]
temp_dates_4                     object
dtype: object

# 2) Extract date related columns

In [57]:
#Changed
date_column_pattern = r"(?i)\b.*(?:date|dob|doj|do[a-z]*).*\b" 

#takes in account all 'Do' related abbrevations like- DoB, DoJ, DoR (date of registration),DoS (date of subscription)...etc,,,,, case insensitive

In [58]:
columns = df.columns
columns

Index(['Ord ID', 'customer name', 'Purchase Date', 'Amount', 'Return Date',
       'DoJ', 'doB', 'ord_date', 'Date', 'i am daTE shreedhar', 'dates',
       'date's', 'date1', 'date_1', 'date_with_date_type',
       'date_with_date_type_2', 'x-y_z-date', 'my_x-y-date-is', 'temp_dates',
       'temp_dates_2', 'temp_dates_3', 'temp_dates_4'],
      dtype='object')

In [61]:
all_date_columns = [col for col in columns if re.search(date_column_pattern, col)]
all_date_columns

['Purchase Date',
 'Return Date',
 'DoJ',
 'doB',
 'ord_date',
 'Date',
 'i am daTE shreedhar',
 'dates',
 "date's",
 'date1',
 'date_1',
 'date_with_date_type',
 'date_with_date_type_2',
 'x-y_z-date',
 'my_x-y-date-is',
 'temp_dates',
 'temp_dates_2',
 'temp_dates_3',
 'temp_dates_4']

# 3) Convert all date data_type columns to there actual or original format and typecast them to object or string.....strptime() will do these 3 task for you

In [62]:
format_mapping = {
        "yyyy\-mm\-dd\ hh:mm:ss": "%Y-%m-%d %H:%M:%S",
        "mm-dd-yy": "%m-%d-%y",
        "[$-F800]dddd\,\ mmmm\ dd\,\ yyyy": "%A, %B %d, %Y"
    }

In [63]:
for col, formats in date_type_columns.items():
    df[col] = df[col].dt.strftime(format_mapping[formats])
#Note dt.strptime will return string,,,,,,,,,,,if you use pd.to_date() to convert columns to date type,,then you lose the formatting again..
#but while storing or exporting the file you need this string only to retain original format,,otherwise pandas will export the date type column in y-m-d format only.

In [64]:
df

,Ord ID,customer name,Purchase Date,Amount,Return Date,DoJ,doB,ord_date,Date,i am daTE shreedhar,...,date1,date_1,date_with_date_type,date_with_date_type_2,x-y_z-date,my_x-y-date-is,temp_dates,temp_dates_2,temp_dates_3,temp_dates_4
0,1,jay,14-10-2021,100,2022-13-12,2015-10-13,13-10-2002,1-13-2020,1-1-2020,1-1-2020,...,None,None,2024-09-13 12:28:36,2024-09-13 12:28:36.614943,None,None,12-13-24,12-13-2024,"Friday, December 13, 2024",12-13-2024 12:28:37
1,2,shree,13-2-2020,200,2021-10-11,2016-10-14,14-10-2002,12-10-2020,2-1-2020,None,...,None,None,2024-09-14 12:28:36,2024-09-14 12:28:36.614943,None,None,12-14-24,12-13-2024,"Saturday, December 14, 2024",12-13-2024 12:28:38
2,3,ram,1-10-2019,300,2020-1-1,2017-10-16,17-10-2002,5-10-2021,3-10-2020,2-1-2020,...,10-12-2002,None,2024-09-15 12:28:36,2024-09-15 12:28:36.614943,None,None,12-15-24,12-13-2024,"Sunday, December 15, 2024",12-13-2024 12:28:39


In [65]:
df.dtypes

Ord ID                    int64
customer name            object
Purchase Date            object
Amount                    int64
Return Date              object
DoJ                      object
doB                      object
ord_date                 object
Date                     object
i am daTE shreedhar      object
dates                    object
date's                   object
date1                    object
date_1                   object
date_with_date_type      object
date_with_date_type_2    object
x-y_z-date               object
my_x-y-date-is           object
temp_dates               object
temp_dates_2             object
temp_dates_3             object
temp_dates_4             object
dtype: object

# 4) Extracting date formats of each date column. Include 1st occurence format if particular date column follows more than 1 date format

In [72]:
def find_date_format(date_str):
    # Define a list of common date formats to check
    possible_formats = [
        '%d-%m-%Y',  # Day-Month-Year ⭐
        '%m-%d-%Y',  # Month-Day-Year 
        '%Y-%m-%d',  # Year-Month-Day #By default pandas read and write "date" type column in this format....⭐
        '%Y-%d-%m',  # year/day/month
        "%Y-%m-%d %H:%M:%S",
        "%d-%m-%Y %H:%M:%S",
        "%m-%d-%Y %H:%M:%S",
        "%A, %B %d, %Y",
        '%m-%d-%y',
        "%Y-%m-%d %H:%M:%S.%f",
    ]
    
    #"Year" never comes in middle...in some scenario "Day" also not come in middle,,,,Majority times "Month" is there in middle
    # "d-m-y" and "y-m-d" are most common
    
    # Try each format
    for date_format in possible_formats:
        try:
            # If date_str can be parsed using the current format, return the format
            datetime.strptime(date_str, date_format)
            return date_format
        except ValueError:
            continue
    
    # If none of the formats work, return None
    return None

In [73]:
column_with_their_first_non_null_occurence_date = dict()
column_with_their_first_non_null_occurence_date

{}

In [74]:
for date_col in all_date_columns:
    if pd.isnull(df[date_col].iloc[0]) and len(df[date_col].value_counts().index) != 0:
        column_with_their_first_non_null_occurence_date[date_col] = df[date_col].value_counts().index[0]
    else:
        column_with_their_first_non_null_occurence_date[date_col] = df[date_col].iloc[0]
column_with_their_first_non_null_occurence_date

{'Purchase Date': '14-10-2021',
 'Return Date': '2022-13-12',
 'DoJ': '2015-10-13',
 'doB': '13-10-2002',
 'ord_date': '1-13-2020',
 'Date': '1-1-2020',
 'i am daTE shreedhar': '1-1-2020',
 'dates': '10-12-2002',
 "date's": '13-12-2002',
 'date1': '10-12-2002',
 'date_1': None,
 'date_with_date_type': '2024-09-13 12:28:36',
 'date_with_date_type_2': '2024-09-13 12:28:36.614943',
 'x-y_z-date': None,
 'my_x-y-date-is': None,
 'temp_dates': '12-13-24',
 'temp_dates_2': '12-13-2024',
 'temp_dates_3': 'Friday, December 13, 2024',
 'temp_dates_4': '12-13-2024 12:28:37'}

In [75]:
column_with_their_first_occurence_date_format= dict()
column_with_their_first_occurence_date_format

{}

In [76]:
for col, date in column_with_their_first_non_null_occurence_date.items():
    if not pd.isnull(date):
        column_with_their_first_occurence_date_format[col] = find_date_format(date)
    else:
        column_with_their_first_occurence_date_format[col] = ''

In [77]:
column_with_their_first_occurence_date_format

{'Purchase Date': '%d-%m-%Y',
 'Return Date': '%Y-%d-%m',
 'DoJ': '%Y-%m-%d',
 'doB': '%d-%m-%Y',
 'ord_date': '%m-%d-%Y',
 'Date': '%d-%m-%Y',
 'i am daTE shreedhar': '%d-%m-%Y',
 'dates': '%d-%m-%Y',
 "date's": '%d-%m-%Y',
 'date1': '%d-%m-%Y',
 'date_1': '',
 'date_with_date_type': '%Y-%m-%d %H:%M:%S',
 'date_with_date_type_2': '%Y-%m-%d %H:%M:%S.%f',
 'x-y_z-date': '',
 'my_x-y-date-is': '',
 'temp_dates': '%m-%d-%y',
 'temp_dates_2': '%m-%d-%Y',
 'temp_dates_3': '%A, %B %d, %Y',
 'temp_dates_4': '%m-%d-%Y %H:%M:%S'}

# 5) Converting all date related columns to date_time type

In [78]:
for col in all_date_columns:
    df[col] = pd.to_datetime(df[col], format=column_with_their_first_occurence_date_format[col])

In [80]:
df
# As u see when we type_cast date related column to date data_type then it will by-defaultly presented in YYYY-MM-DD format..

,Ord ID,customer name,Purchase Date,Amount,Return Date,DoJ,doB,ord_date,Date,i am daTE shreedhar,...,date1,date_1,date_with_date_type,date_with_date_type_2,x-y_z-date,my_x-y-date-is,temp_dates,temp_dates_2,temp_dates_3,temp_dates_4
0,1,jay,2021-10-14,100,2022-12-13,2015-10-13,2002-10-13,2020-01-13,2020-01-01,2020-01-01,...,NaT,NaT,2024-09-13 12:28:36,2024-09-13 12:28:36.614943,NaT,NaT,2024-12-13,2024-12-13,2024-12-13,2024-12-13 12:28:37
1,2,shree,2020-02-13,200,2021-11-10,2016-10-14,2002-10-14,2020-12-10,2020-01-02,NaT,...,NaT,NaT,2024-09-14 12:28:36,2024-09-14 12:28:36.614943,NaT,NaT,2024-12-14,2024-12-13,2024-12-14,2024-12-13 12:28:38
2,3,ram,2019-10-01,300,2020-01-01,2017-10-16,2002-10-17,2021-05-10,2020-10-03,2020-01-02,...,2002-12-10,NaT,2024-09-15 12:28:36,2024-09-15 12:28:36.614943,NaT,NaT,2024-12-15,2024-12-13,2024-12-15,2024-12-13 12:28:39


In [81]:
df.dtypes

Ord ID                            int64
customer name                    object
Purchase Date            datetime64[ns]
Amount                            int64
Return Date              datetime64[ns]
DoJ                      datetime64[ns]
doB                      datetime64[ns]
ord_date                 datetime64[ns]
Date                     datetime64[ns]
i am daTE shreedhar      datetime64[ns]
dates                    datetime64[ns]
date's                   datetime64[ns]
date1                    datetime64[ns]
date_1                   datetime64[ns]
date_with_date_type      datetime64[ns]
date_with_date_type_2    datetime64[ns]
x-y_z-date               datetime64[ns]
my_x-y-date-is           datetime64[ns]
temp_dates               datetime64[ns]
temp_dates_2             datetime64[ns]
temp_dates_3             datetime64[ns]
temp_dates_4             datetime64[ns]
dtype: object

In [82]:
df.columns

Index(['Ord ID', 'customer name', 'Purchase Date', 'Amount', 'Return Date',
       'DoJ', 'doB', 'ord_date', 'Date', 'i am daTE shreedhar', 'dates',
       'date's', 'date1', 'date_1', 'date_with_date_type',
       'date_with_date_type_2', 'x-y_z-date', 'my_x-y-date-is', 'temp_dates',
       'temp_dates_2', 'temp_dates_3', 'temp_dates_4'],
      dtype='object')

# How openpyxl works

1. Load the workbook and select the worksheet:
python
Copy code
file_path = 'data.xlsx'
wb = load_workbook(file_path, data_only=True)
ws = wb.active
file_path: Specifies the path to the Excel file.
load_workbook(file_path, data_only=True): Loads the Excel workbook using openpyxl. The data_only=True argument ensures that the values returned are those stored in the cells, not the formulas.
wb.active: Selects the currently active worksheet from the workbook (ws represents the active sheet).
2. Create an empty dictionary to hold the data:
python
Copy code
data = {}
data: Initializes an empty dictionary where each key will correspond to a column header, and the values will be lists holding the column’s data.
3. Fetch the headers:
python
Copy code
for cell in ws[1]:
    data[cell.value] = []
ws[1]: Refers to the first row in the worksheet, which usually contains the headers (column names).
The loop iterates through each cell in the first row, extracts its value (the header), and creates an empty list for each header in the data dictionary.
4. Initialize a dictionary to store date formats for columns:
python
Copy code
date_formats = {}
date_formats: Creates an empty dictionary that will store the format of each column. The key is the column header, and the value is the date format if the column contains dates.
5. Iterate over columns to find the first non-null value in each:
python
Copy code
for col in ws.iter_cols(min_row=2, values_only=False):
ws.iter_cols(min_row=2, values_only=False): Iterates through all the columns, starting from the second row (min_row=2) to skip the header row. The values_only=False parameter allows access to the entire cell object, not just its value, so that date formats can be inspected.
6. For each column, fetch the header and check for the first non-null cell:
python
Copy code
header = ws.cell(row=1, column=col[0].column).value  # Get the column header
ws.cell(row=1, column=col[0].column).value: Retrieves the value of the first row in the current column (which is the header).
python
Copy code
for cell in col:
    if cell.value is not None:  # Check for the first non-null cell
Loops through all the cells in the current column.
cell.value is not None: Skips empty cells and processes the first non-empty one.
7. Check if the first non-null cell is a date:
python
Copy code
if cell.is_date:
    date_formats[header] = cell.number_format  # Store date format
else:
    date_formats[header] = None  # No special format for non-date columns
cell.is_date: This checks if the cell contains a date value.
If the cell is a date, the code retrieves its format (cell.number_format) and stores it in the date_formats dictionary, where the key is the column header.
If the cell is not a date, None is assigned as the format.
python
Copy code
break  # Stop after finding the first non-null value
Breaks out of the loop once the first non-null cell is found, as we are only interested in identifying the format based on the first valid entry.
8. Fetch the data for all rows without applying formats to the DataFrame:
python
Copy code
for row in ws.iter_rows(min_row=2, values_only=False):
ws.iter_rows(min_row=2, values_only=False): Iterates through all rows starting from the second row (to skip the header), retrieving the entire cell object instead of just its value.
python
Copy code
for cell in row:
    header = ws.cell(row=1, column=cell.column).value
    data[header].append(cell.value)
For each cell in the row, the code retrieves its corresponding header (from the first row) and appends the cell's value to the appropriate list in the data dictionary.
9. Convert the dictionary to a pandas DataFrame:
python
Copy code
df = pd.DataFrame(data)
Converts the data dictionary into a pandas DataFrame, where each key in the dictionary becomes a column in the DataFrame, and the values (lists) become the column's data.
10. Print the stored date formats:
python
Copy code
print("Date columns and their formats:", date_formats)
Displays the date formats stored in the date_formats dictionary, showing which columns contain dates and their respective formats.
Summary:
The code loads an Excel file and iterates over its cells to detect date columns and their formats.
It dynamically builds a date_formats dictionary to store the formats of the date columns.
It populates a data dictionary with values from the worksheet and then converts it to a pandas DataFrame.